In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import miceforest as mf

from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

In [ ]:
B1_Lab_1=pd.read_excel("B1_Lab-1.xlsx")
B1_Lab_3=pd.read_excel("B1_Lab-3.xlsx")

In [ ]:
well_B1=B1_Lab_1.merge(B1_Lab_3, how='outer', on='Depth',sort=True,suffixes=('', '_LAB-3')).assign(WELL='Well_B1')
depth_B1=well_B1['Depth']

In [ ]:
well_B1.drop(well_B1.head(114).index, inplace=True)
well_B1.drop([247], axis=0, inplace=True)
well_B1=well_B1.reset_index(drop=True)

In [ ]:
well_B1=well_B1.drop(['WELL','Zr_LAB-3','Nb_LAB-3','Th_LAB-3','Cr_LAB-3','Ga_LAB-3', 'Cu_LAB-3'], axis=1) 

In [ ]:
well_B1=well_B1.drop(['Depth'], axis=1)

### NaN figure

In [ ]:
well_B1_NaN=well_B1.copy()
well_B1_NaN.index += 1

well_B1_NaN.columns=well_B1_NaN.columns.str.replace('_LAB-3','')

plt.figure(figsize=(20, 15), dpi = 500)

colors = ["#6E9FD0", "black"]
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
plt.rcParams['font.family'] = 'arial'

sns.heatmap(well_B1_NaN.isnull(), cbar=False, cmap=cmap, yticklabels=115)

plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14, rotation=0)

plt.show() 

## miceforest No NaN set

In [ ]:
No_NaN=well_B1.dropna()
No_NaN.reset_index(drop=True, inplace=True)

In [ ]:
cust_kernel=mf.ImputationKernel(No_NaN, datasets=10, save_all_iterations=True, random_state=11,
                                variable_schema=No_NaN.columns.to_list(), train_nonmissing=True)

optimal_parametros, losses = cust_kernel.tune_parameters(dataset=0, optimization_steps=15,  random_state=11)

cust_kernel.mice(6,variable_parameters= optimal_parametros)

In [ ]:
new_data_imp=cust_kernel.impute_new_data(well_B1)
MF_Imp=new_data_imp.complete_data()
MF_Imp.to_excel("Well_B1_Imp.xlsx")